In [ ]:
def RunSurfaceTensionCalc(nsteps=5000, equil_steps=5000, hdf5_log='surface_tension_log.hdf5'):
    L = 50.0
    kT = 0.8

    # Initialize positions with two-phase setup
    positions = FillBoxTwoPhase(
        xlim=np.array([-0.5 * L, 0.5 * L]),
        ylim=np.array([-0.5 * L, 0.5 * L]),
        zlim=np.array([-0.9 * L, 0.9 * L]),
        rho_liquid=0.85,
        rho_vapor=0.01
    )
    N_particles = positions.shape[0]

    # Create GSD frame
    frame = gsd.hoomd.Frame()
    frame.configuration.box = [L, L, 2 * L, 0, 0, 0]  # Box with periodic boundaries
    frame.particles.types = ['A']
    frame.particles.N = N_particles
    frame.particles.position = positions

    # Initialize simulation
    device = hoomd.device.CPU()
    simulation = hoomd.Simulation(device=device, seed=1)
    simulation.create_state_from_snapshot(frame)
    simulation.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=kT)

    # Select LJ model
    lj = SelectLJModel(rcut=3.0)
    
    # Set always compute pressure
    simulation.always_compute_pressure = True

    # Setup integrator
    integrator = hoomd.md.Integrator(dt=0.001)  # Reduced time step for stability
    integrator.forces.append(lj)
    nvt = hoomd.md.methods.ConstantVolume(
        filter=hoomd.filter.All(),
        thermostat=hoomd.md.methods.thermostats.Bussi(kT=kT)
    )
    integrator.methods.append(nvt)
    simulation.operations.integrator = integrator

    # Compute thermodynamic quantities
    thermo = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())
    simulation.operations.computes.append(thermo)

    print("Starting equilibration for surface tension calculation...")
    simulation.run(equil_steps)
    print("Equilibration complete. Starting production run...")

    # Setup HDF5 logger
    with h5py.File(hdf5_log, 'w') as h5file:
        grp = h5file.create_group("pressure_tensor")
        grp.create_dataset("P_xx", shape=(0,), maxshape=(None,), dtype='f')
        grp.create_dataset("P_yy", shape=(0,), maxshape=(None,), dtype='f')
        grp.create_dataset("P_zz", shape=(0,), maxshape=(None,), dtype='f')
        
        for step in range(0, nsteps, 100):
            simulation.run(100)
            pressure_tensor = np.array(thermo.pressure_tensor)
            P_xx, P_yy, P_zz = pressure_tensor[0], pressure_tensor[1], pressure_tensor[2]
            
            # Corrected resize calls: pass integer size when axis is specified
            grp["P_xx"].resize(grp["P_xx"].shape[0] + 1, axis=0)
            grp["P_xx"][-1] = P_xx
            grp["P_yy"].resize(grp["P_yy"].shape[0] + 1, axis=0)
            grp["P_yy"][-1] = P_yy
            grp["P_zz"].resize(grp["P_zz"].shape[0] + 1, axis=0)
            grp["P_zz"][-1] = P_zz
            print(f"Logged step {step + 100}")

    print("Production run complete. Calculating surface tension...")

    # Reload simulation to compute surface tension
    # Note: Alternatively, you can process the HDF5 file directly without reloading the simulation
    surface_tension = 0.0
    with h5py.File(hdf5_log, 'r') as h5file:
        P_xx = h5file["pressure_tensor/P_xx"][:]
        P_yy = h5file["pressure_tensor/P_yy"][:]
        P_zz = h5file["pressure_tensor/P_zz"][:]
        P_xx_avg = np.mean(P_xx)
        P_yy_avg = np.mean(P_yy)
        P_zz_avg = np.mean(P_zz)
        surface_tension = (P_zz_avg - 0.5 * (P_xx_avg + P_yy_avg)) * (L / 2.0)
    
    print(f"Calculated surface tension: {surface_tension}")
    
    #compute density ratio
    boundary_z = 0.0  
    density_liquid, density_vapor, ratio = compute_density_ratio(simulation, boundary_z)
    print(f"Liquid density: {density_liquid}")
    print(f"Vapor density: {density_vapor}")
    print(f"Density ratio (liquid/vapor): {ratio}")

    return {
        "final_surface_tension": surface_tension,
        "density_ratio": ratio,
        "hdf5_log": hdf5_log
    }


def load_pressure_tensor(hdf5_log): 
    """
    Loads pressure tensor data from an HDF5 log file.
    """
    with h5py.File(hdf5_log, 'r') as h5file:
        P_xx = h5file["pressure_tensor/P_xx"][:]
        P_yy = h5file["pressure_tensor/P_yy"][:]
        P_zz = h5file["pressure_tensor/P_zz"][:]
    return P_xx, P_yy, P_zz

if __name__ == "__main__":
    surface_results = RunSurfaceTensionCalc(
        nsteps=5000,
        equil_steps=5000,
        hdf5_log='surface_tension_log.hdf5'
    )
    print(f"Calculated Surface Tension: {surface_results['final_surface_tension']}")
